In [1]:
from diffpy.Structure import loadStructure
from diffpy.srreal.pdfcalculator import DebyePDFCalculator, PDFCalculator
from matplotlib.pyplot import plot, show
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

In [ ]:
def catalogue_to_arrays(catalogue, number_of_best_structures):
    '''This function reads the prepared catalogue of structures and returns
    only user-provided number of best structures based on Rw value of the PDF fits.'''
    with open(catalogue, 'r') as f:
        strings = f.readlines()
    filtered_strings = [s for s in strings if float(s.split()[1]) <= 0.95]
    sorted_strings = sorted(filtered_strings, key=lambda x: float(x.split()[1]))
    arrays = []
    for s in sorted_strings:
        # Split the string into a list of numbers
        numbers = list(map(float, s.split()))
        arrays.append(np.array(numbers))
    return arrays[:number_of_best_structures]


def modify_xyz_files(arrays, xyz_filename, new_dir):
    '''This function takes the selected best structures and the initial xyz file with the starting model
    and creates a series of xyz files with those best structures.'''
    # Create the new directory if it doesn't exist
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    # Open the input xyz file
    with open(xyz_filename, 'r') as f:
        lines = f.readlines()

    # Get the number of atoms from the xyz file
    num_atoms = int(lines[0])

    # Iterate over the arrays
    for array in arrays:
        # Create a list to store the modified xyz lines
        new_lines = []

        # Append the header to the new lines list
        new_lines.append(f"{num_atoms}\n")
        new_lines.append(lines[1])

        # Iterate over the atoms in the xyz file
        for i, line in enumerate(lines[2:]):
            # If the index is less than the length of the array, process the atom according to the array value
            if i < len(array):
                if array[i] == 1:
                    new_lines.append(line)
            # If the index is greater than or equal to the length of the array, retain the atom
            else:
                new_lines.append(line)

        # Compute the new xyz filename using the first two values of the initial array
        new_filename = "{:.2f}_{:.0f}.xyz".format(array[1], array[0])

        # Join the new directory and the new filename
        new_path = os.path.join(new_dir, new_filename)

        # Write the modified xyz file
        with open(new_path, 'w') as f:
            f.writelines(new_lines)

def filter_atoms(path, new_directory, threshold_distance):
    '''This function takes the xyz files created on a previous step and deletes
    all non-bonded atoms based on the user-provided treshold value.'''
    # Create new directory if it doesn't exist
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    
    with open(path, 'r') as f:
        lines = f.readlines()
        # Prepare new filename and write the new file
        new_filename = f"{path.split('/')[-1]}"
        new_file = os.path.join(new_directory, new_filename)
        with open(new_file, 'w') as new_f:
            # write the header
            new_f.write(lines[0])
            new_f.write(lines[1])
            # Create a list to store the Th coordinates
            Th_coordinates = []
            for line in lines[2:]:
                fields = line.split()
                if fields and fields[0] == "Th":
                    new_f.write(line)
                    Th_coordinates.append([float(fields[1]), float(fields[2]), float(fields[3])])
                elif fields and fields[0] == "O":
                    O_coordinate = [float(fields[1]), float(fields[2]), float(fields[3])]
                    for Th_coordinate in Th_coordinates:
                        distance = np.linalg.norm(np.array(Th_coordinate) - np.array(O_coordinate))
                        if distance < threshold_distance:
                            new_f.write(line)
                            break

In [39]:
def make_xyz_catalogue(catalogue, initial_xyz, treshold_ThO, treshold_ThTh):
    
    with open(catalogue, 'r') as f:
        strings = f.readlines()
    arrays = []
    for s in strings:
        # Split the string into a list of numbers
        numbers = list(map(float, s.split()))
        arrays.append(np.array(numbers))

    with open(initial_xyz, 'r') as f:
        lines = f.readlines()

    #print(lines)
    # Get the number of atoms from the xyz file
    num_atoms = int(lines[0])

    # Iterate over the arrays
    for array, k in zip(arrays, range(len(arrays)+1)):
        # Create a list to store the modified xyz lines
        new_lines = [f"{num_atoms}\n", lines[1]]

        # Iterate over the atoms in the xyz file
        for i, line in enumerate(lines[2:]):
            # If the index is less than the length of the array, process the atom according to the array value
            if i < len(array) and array[i] == 1 or i >= len(array):
                new_lines.append(line)
        #print(new_lines)

        Th_coordinates = []
        for line in new_lines[2:]:
            fields = line.split()
            if fields and fields[0] == "Th":
                Th_coordinates.append([float(fields[1]), float(fields[2]), float(fields[3])])

        Th_groups = []
        while Th_coordinates:
            Th_group = [Th_coordinates.pop()]
            for Th_coordinate in Th_coordinates:
                distance = np.linalg.norm(np.array(Th_group[0]) - np.array(Th_coordinate))
                if distance < treshold_ThTh:
                    Th_group.append(Th_coordinate)
                    Th_coordinates.remove(Th_coordinate)
            Th_groups.append(Th_group)

        if Th_groups:

            largest_Th_group = max(Th_groups, key=lambda x: len(x))
            
            with open(f'{initial_xyz}_{k}.xyz', 'w') as new_f:
                new_f.write(new_lines[0])
                new_f.write(new_lines[1])
                for Th_coordinate in largest_Th_group:
                    new_f.write("Th " + " ".join([str(x) for x in Th_coordinate]) + "\n")
                for line in lines[2:]:
                    fields = line.split()
                    if fields and fields[0] == "O":
                        O_coordinate = [float(fields[1]), float(fields[2]), float(fields[3])]
                        for Th_coordinate in largest_Th_group:
                            distance = np.linalg.norm(np.array(Th_coordinate) - np.array(O_coordinate))
                            if distance < treshold_ThO:
                                new_f.write(line)
                                break


In [46]:
os.chdir('/Users/dimitrygrebenyuk/Yandex.Disk.localized/Working/PDF/Refinements/PDF-Cluster-Prediction/th_clusters')

In [47]:
make_xyz_catalogue(catalogue='Th22_03_spot1_0001_0000_summed_bsub_tmean_gs2.txt', initial_xyz='40.xyz', 
                   treshold_ThO=3.0, treshold_ThTh=5.2)

In [51]:
directory = "/Users/dimitrygrebenyuk/Yandex.Disk.localized/Working/PDF/Refinements/PDF-Cluster-Prediction/th_clusters"

for filename, i in zip(os.listdir(directory), range(10001)):
    path = os.path.join(directory, filename)
    if path.startswith('/Users/dimitrygrebenyuk/Yandex.Disk.localized/Working/PDF/Refinements/PDF-Cluster-Prediction/th_clusters/40'):
        with open(path, 'r', encoding='ISO-8859-1') as f:
            lines = f.readlines()
            Th_count = sum(1 for line in lines if line.startswith("Th"))
            new_filename = f"{Th_count}_{i}.xyz"
            new_path = os.path.join(directory, new_filename)
            os.rename(path, new_path)